In [1]:
import fitz
import os
import numpy as np
import json
import re
from openai import OpenAI
import torch
from sentence_transformers import SentenceTransformer, SimilarityFunction
from tqdm import tqdm

/home/patrick/rag_from_scratch/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the pre-trained model
embedder = SentenceTransformer("all-MiniLM-L6-v2") # Fazer o embedding usando esse

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

In [4]:
def chunk_text(text, chunk_size, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []

    for i in range(0, len(text), chunk_size-overlap):
        chunks.append(text[i:i+chunk_size])
    return chunks

In [5]:
def create_embeddings(text):
    response = embedder.encode(text)
    return response

In [6]:
class SimpleVectorStore:
    """
    A simple vector store implementation using NumPy.
    """
    def __init__(self):
        """
        Initialize the vector store.
        """
        self.vectors = []  # List to store embedding vectors
        self.texts = []  # List to store original texts
        self.metadata = []  # List to store metadata for each text
    
    def add_item(self, text, embedding, metadata=None):
        """
        Add an item to the vector store.

        Args:
        text (str): The original text.
        embedding (List[float]): The embedding vector.
        metadata (dict, optional): Additional metadata.
        """
        self.vectors.append(np.array(embedding))  # Convert embedding to numpy array and add to vectors list
        self.texts.append(text)  # Add the original text to texts list
        self.metadata.append(metadata or {})  # Add metadata to metadata list, default to empty dict if None
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        Find the most similar items to a query embedding.

        Args:
        query_embedding (List[float]): Query embedding vector.
        k (int): Number of results to return.
        filter_func (callable, optional): Function to filter results.

        Returns:
        List[Dict]: Top k most similar items with their texts and metadata.
        """
        if not self.vectors:
            return []  # Return empty list if no vectors are stored
        
        # Convert query embedding to numpy array
        query_vector = np.array(query_embedding)
        
        # Calculate similarities using cosine similarity
        similarities = []
        for i, vector in enumerate(self.vectors):
            # Apply filter if provided
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # Calculate cosine similarity
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # Append index and similarity score
        
        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Return top k results
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # Add the text
                "metadata": self.metadata[idx],  # Add the metadata
                "similarity": score  # Add the similarity score
            })
        
        return results  # Return the list of top k results

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    Process a document for use with adaptive retrieval.

    Args:
    pdf_path (str): Path to the PDF file.
    chunk_size (int): Size of each chunk in characters.
    chunk_overlap (int): Overlap between chunks in characters.

    Returns:
    Tuple[List[str], SimpleVectorStore]: Document chunks and vector store.
    """
    # Extract text from the PDF file
    print("Extracting text from PDF...")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    # Chunk the extracted text
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # Create embeddings for the text chunks
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(chunks)
    
    # Initialize the vector store
    store = SimpleVectorStore()
    
    # Add each chunk and its embedding to the vector store with metadata
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    
    # Return the chunks and the vector store
    return chunks, store

In [9]:
def determine_if_retrieval_needed(query):
    """
    Determines if retrieval is necessary for the given query.
    
    Args:
        query (str): User query
        
    Returns:
        bool: True if retrieval is needed, False otherwise
    """
    # System prompt to instruct the AI on how to determine if retrieval is necessary
    system_prompt = """You are an AI assistant that determines if retrieval is necessary to answer a query.
    For factual questions, specific information requests, or questions about events, people, or concepts, answer "Yes".
    For opinions, hypothetical scenarios, or simple queries with common knowledge, answer "No".
    Answer with ONLY "Yes" or "No"."""

    # User prompt containing the query
    user_prompt = f"Query: {query}\n\nIs retrieval necessary to answer this query accurately?"
    
    # Generate response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()
    
    # Return True if the answer contains "yes", otherwise return False
    return "yes" in answer

In [18]:
def evaluate_relevance(query, context):
    """
    Evaluates the relevance of a context to the query.
    
    Args:
        query (str): User query
        context (str): Context text
        
    Returns:
        str: 'relevant' or 'irrelevant'
    """
    # System prompt to instruct the AI on how to determine document relevance
    system_prompt = """You are an AI assistant that determines if a document is relevant to a query.
    Consider whether the document contains information that would be helpful in answering the query.
    Answer with ONLY "Relevant" or "Irrelevant"."""

    # Truncate context if it is too long to avoid exceeding token limits
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # User prompt containing the query and the document content
    user_prompt = f"""Query: {query}
    Document content:
    {context}

    Is this document relevant to the query? Answer with ONLY "Relevant" or "Irrelevant".
    """
    
    # Generate response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()
    
    return answer  # Return the relevance evaluation

In [19]:
def assess_support(response, context):
    """
    Assesses how well a response is supported by the context.
    
    Args:
        response (str): Generated response
        context (str): Context text
        
    Returns:
        str: 'fully supported', 'partially supported', or 'no support'
    """
    # System prompt to instruct the AI on how to evaluate support
    system_prompt = """You are an AI assistant that determines if a response is supported by the given context.
    Evaluate if the facts, claims, and information in the response are backed by the context.
    Answer with ONLY one of these three options:
    - "Fully supported": All information in the response is directly supported by the context.
    - "Partially supported": Some information in the response is supported by the context, but some is not.
    - "No support": The response contains significant information not found in or contradicting the context.
    """

    # Truncate context if it is too long to avoid exceeding token limits
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # User prompt containing the context and the response to be evaluated
    user_prompt = f"""Context:
    {context}

    Response:
    {response}

    How well is this response supported by the context? Answer with ONLY "Fully supported", "Partially supported", or "No support".
    """
    
    # Generate response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()
    
    return answer  # Return the support assessment

In [20]:
def rate_utility(query, response):
    """
    Rates the utility of a response for the query.
    
    Args:
        query (str): User query
        response (str): Generated response
        
    Returns:
        int: Utility rating from 1 to 5
    """
    # System prompt to instruct the AI on how to rate the utility of the response
    system_prompt = """You are an AI assistant that rates the utility of a response to a query.
    Consider how well the response answers the query, its completeness, correctness, and helpfulness.
    Rate the utility on a scale from 1 to 5, where:
    - 1: Not useful at all
    - 2: Slightly useful
    - 3: Moderately useful
    - 4: Very useful
    - 5: Exceptionally useful
    Answer with ONLY a single number from 1 to 5."""

    # User prompt containing the query and the response to be rated
    user_prompt = f"""Query: {query}
    Response:
    {response}

    Rate the utility of this response on a scale from 1 to 5:"""
    
    # Generate the utility rating using the OpenAI client
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # Extract the rating from the model's response
    rating = response.choices[0].message.content.strip()
    
    # Extract just the number from the rating
    rating_match = re.search(r'[1-5]', rating)
    if rating_match:
        return int(rating_match.group())  # Return the extracted rating as an integer
    
    return 3  # Default to middle rating if parsing fails


In [21]:
# Define the path to the PDF file
pdf_path = "AI_Information.pdf"

# Process the document (extract text, create chunks, generate questions, build vector store)
vector_store = process_document(
    pdf_path, 
    chunk_size=1000)

Extracting text from PDF...
Chunking text...
Created 42 text chunks
Creating embeddings for chunks...
Added 42 chunks to the vector store


In [22]:
def semantic_search(query, vector_store, k=5):
    """
    Performs semantic search using the query and vector store.

    Args:
    query (str): The search query.
    vector_store (SimpleVectorStore): The vector store to search in.
    k (int): Number of results to return.

    Returns:
    List[Dict]: Top k most relevant items.
    """
    # Create embedding for the query
    query_embedding_response = create_embeddings(query)
    query_embedding = query_embedding_response
    
    # Search the vector store
    results = vector_store.similarity_search(query_embedding, k=k)
    
    return results

In [23]:
def generate_response(query, context=None):
    """
    Generates a response based on the query and optional context.
    
    Args:
        query (str): User query
        context (str, optional): Context text
        
    Returns:
        str: Generated response
    """
    # System prompt to instruct the AI on how to generate a helpful response
    system_prompt = """You are a helpful AI assistant. Provide a clear, accurate, and informative response to the query."""
    
    # Create the user prompt based on whether context is provided
    if context:
        user_prompt = f"""Context:
        {context}

        Query: {query}

        Please answer the query based on the provided context.
        """
    else:
        user_prompt = f"""Query: {query}
        
        Please answer the query to the best of your ability."""
    
    # Generate the response using the OpenAI client
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    
    # Return the generated response text
    return response.choices[0].message.content.strip()

In [24]:
def self_rag(query, vector_store, top_k=3):
    """
    Implements the complete Self-RAG pipeline.
    
    Args:
        query (str): User query
        vector_store (SimpleVectorStore): Vector store containing document chunks
        top_k (int): Number of documents to retrieve initially
        
    Returns:
        dict: Results including query, response, and metrics from the Self-RAG process
    """
    
    # Step 1: Determine if retrieval is necessary
    retrieval_needed = determine_if_retrieval_needed(query)
    
    # Initialize metrics to track the Self-RAG process
    metrics = {
        "retrieval_needed": retrieval_needed,
        "documents_retrieved": 0,
        "relevant_documents": 0,
        "response_support_ratings": [],
        "utility_ratings": []
    }
    
    best_response = None
    best_score = -1
    
    if retrieval_needed:
        # Step 2: Retrieve documents
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)
        metrics["documents_retrieved"] = len(results)
        
        # Step 3: Evaluate relevance of each document
        relevant_contexts = []
        
        for i, result in enumerate(results):
            context = result["text"]
            relevance = evaluate_relevance(query, context)
            print(f"Document {i+1} relevance: {relevance}")
            
            if relevance == "relevant":
                relevant_contexts.append(context)
        
        metrics["relevant_documents"] = len(relevant_contexts)
        
        if relevant_contexts:
            # Step 4: Process each relevant context
            for i, context in enumerate(relevant_contexts):
                
                # Generate response based on the context
                response = generate_response(query, context)
                
                # Assess how well the response is supported by the context
                support_rating = assess_support(response, context)
                metrics["response_support_ratings"].append(support_rating)
                
                # Rate the utility of the response
                utility_rating = rate_utility(query, response)
                metrics["utility_ratings"].append(utility_rating)
                
                # Calculate overall score (higher for better support and utility)
                support_score = {
                    "fully supported": 3, 
                    "partially supported": 1, 
                    "no support": 0
                }.get(support_rating, 0)
                
                overall_score = support_score * 5 + utility_rating
                print(f"Overall score: {overall_score}")
                
                # Keep track of the best response
                if overall_score > best_score:
                    best_response = response
                    best_score = overall_score
                    print("New best response found!")
        
        # If no relevant contexts were found or all responses scored poorly
        if not relevant_contexts or best_score <= 0:
            best_response = generate_response(query)
    else:
        # No retrieval needed, generate directly
        best_response = generate_response(query)
    
    # Final metrics
    metrics["best_score"] = best_score
    metrics["used_retrieval"] = retrieval_needed and best_score > 0
        
    return {
        "query": query,
        "response": best_response,
        "metrics": metrics
    }

In [25]:
query = "what are the concerns about AI?"

In [26]:
chunks, vector_store = process_document(pdf_path)
    
# Initialize collection for storing comparison results

# Process each test query with both retrieval methods

# Create embedding for the query
query_embedding = create_embeddings(query)

rag_result = self_rag(query, vector_store, top_k=3)


Extracting text from PDF...
Chunking text...
Created 42 text chunks
Creating embeddings for chunks...
Added 42 chunks to the vector store
Document 1 relevance: relevant
Document 2 relevance: relevant
Document 3 relevance: relevant
Overall score: 20
New best response found!
Overall score: 20
Overall score: 20


In [29]:
print(rag_result['response'])

The concerns about AI include:

1. Lack of transparency and explainability in decision-making, particularly in deep learning models, leading to difficulty in understanding how decisions are reached.
2. Privacy and data security issues due to the reliance of AI systems on large amounts of data, raising concerns about protecting sensitive information and ensuring responsible data handling.
3. Job displacement concerns, particularly in industries with repetitive or routine tasks, due to the automation capabilities of AI.
4. Questions about control, accountability, and potential unintended consequences as AI systems become more autonomous, highlighting the need for clear guidelines and ethical frameworks for AI development and deployment.
5. The potential for the weaponization of AI, raising concerns about the misuse of AI technology for harmful purposes.
